### Задача 1: сравнение предложений  

Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

* кошки (животные)  
* UNIX-утилита cat для вывода содержимого файлов  
* версии операционной системы OS X, названные в честь семейства кошачьих  

Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

Выполните следующие шаги:  
* 1. Скачайте файл с предложениями (sentences.txt).
* 2. Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().

In [259]:
file_obj = open('sentences.txt', 'r')

In [260]:
sentences_list = []

for line in file_obj.readlines():
    sentences_list.append(line.strip().lower())
    
file_obj.close()    

* Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.

In [261]:
import re

words_list = []
for line in sentences_list:    
    for word in re.split('[^a-z]', line):
        if word != '':
            words_list.append(word)

* Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.

In [262]:
words_set = set(words_list)
words_index = dict()
i = 0

for word in words_set:
    words_index[i] = word
    i = i + 1
    

* Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.

In [263]:
import numpy as np
words_matrix = np.zeros((len(sentences_list),len(words_set)), dtype = int)
print words_matrix.shape

words_in_sentences = list()
for line in sentences_list:
    words_in_sentences.append( re.split('[^a-z]', line) )

for i in range(0, len(sentences_list)):
    for j in range(0, len(words_set)):
        for w in words_in_sentences[i]:
            if w == words_index[j]:
                words_matrix[i,j] = words_matrix[i, j] + 1
words_matrix                

(22, 254)


array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

* Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.

In [287]:
import scipy.spatial as spatial

distance_vector = [spatial.distance.cosine(words_matrix[0, :], words_matrix[0, :])]

for i in range(1, len(sentences_list)):
    dist = spatial.distance.cosine(words_matrix[0, :], words_matrix[i, :])
    distance_vector.append(dist)

min_val1 = 1
min_idx1 = 0
for d in range(1, len(distance_vector)):
    if distance_vector[d] < min_val1:
        min_val1 = distance_vector[d]
        min_idx1 = d
        
min_val2 = 1
min_idx2 = 0
for d in range(1, len(distance_vector)):
    if d == min_idx1:
        continue
    if distance_vector[d] < min_val2:
        min_val2 = distance_vector[d]
        min_idx2 = d

    
for p in distance_vector:
    print p
print ""    
print min_val1
print min_idx1
print min_val2
print min_idx2

0.0
0.9527544408738466
0.8644738145642124
0.8951715163278082
0.7770887149698589
0.9402385695332803
0.7327387580875756
0.9258750683338899
0.8842724875284311
0.9055088817476932
0.8328165362273942
0.8804771390665607
0.8396432548525454
0.8703592552895671
0.8740118423302576
0.9442721787424647
0.8406361854220809
0.956644501523794
0.9442721787424647
0.8885443574849294
0.8427572744917122
0.8250364469440588

0.7327387580875756
6
0.7770887149698589
4


* Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).

In [290]:
file_obj = open('submission-1.txt', 'w')
string = str(min_val1) + " " + str(min_val2)
file_obj.write(string)
file_obj.close()